<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/annotated_diffusion_%E6%B3%A8%E9%87%88%E4%BB%98%E3%81%8D_%E6%99%AE%E5%8F%8A%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>
	注釈付き 普及モデル
</h1>


<div class="author-card">
    <a href="/nielsr">
        <img class="avatar avatar-user" src="https://avatars.githubusercontent.com/u/48327001?v=4" width="100" title="Gravatar">
        <div class="bfc">
            <code>nielsr</code>
            <span class="fullname">Niels Rogge</span>
        </div>
    </a>
    <a href="/kashif">
        <img class="avatar avatar-user" src="https://avatars.githubusercontent.com/u/8100?v=4" width="100" title="Gravatar">
        <div class="bfc">
            <code>kashif</code>
            <span class="fullname">Kashif Rasul</span>
        </div>
    </a>
    
</div>

<script async defer src="https://unpkg.com/medium-zoom-element@0/dist/medium-zoom-element.min.js"></script>



このブログでは、DenoisingDiffusion ProbabilisticModels（DDPM、ノイズ除去拡散確率モデル ,拡散モデル、スコアベース生成モデル、または単に[オートエンコーダー](https://benanne.github.io/2022/01/31/diffusion.html)としても知られています）について深く見ていきます。研究者は、**（無）条件画像／音声／動画生成**のために、これらのモデルを使って顕著な結果を出すことができました。代表的な例としては、**OpenAIの[GLIDE](https://arxiv.org/abs/2112.10741)や [DALL-E 2](https://openai.com/dall-e-2/)**、**ハイデルベルク大学の[Latent Diffusion](https://github.com/CompVis/latent-diffusion)**、**Google Brainの[ImageGen](https://imagen.research.google/)など**があります（記事執筆時）。

[(Ho et al., 2020)](https://arxiv.org/abs/2006.11239)によるオリジナルのDDPMの論文を、[Phil Wangの実装](https://github.com/lucidrains/denoising-diffusion-pytorch)-それ自体は[オリジナルのTensorFlowの実装](https://github.com/hojonathanho/diffusion)をベースに、PyTorchでステップバイステップで実装しながら見ていきます。なお、生成モデリングのための拡散の考え方は、実は[(Sohl-Dickstein et al., 2015)](https://arxiv.org/abs/1503.03585)で既に紹介されていました。しかし、このアプローチを独自に改良したのは、[(Song et al., 2019)](https://arxiv.org/abs/1907.05600) (at Stanford University)、そして[(Ho et al., 2020)](https://arxiv.org/abs/2006.11239) (at Google Brain) まででした。

なお、拡散モデルには[いくつかの視点](https://twitter.com/sedielem/status/1530894256168222722?s=20&t=mfv4afx1GcNQU5fZklpACw)がある。ここでは、離散時間（潜在変数モデル）の視点を採用しますが、他の視点もぜひチェックしてみてください。

では、さっそく本題に入りましょう。

まず必要なライブラリをインストールし、インポートします（PyTorchが インストールされていることが前提です）。

<p align="center">
<img src='https://drive.google.com/uc?id=11C3cBUfz7_vrkj_4CWCyePaQyr-0m85_' width=500>
</p>

まず必要なライブラリをインストールし、インポートします（PyTorchが インストールされていることが前提です）。

In [ ]:
!pip install -q -U einops datasets matplotlib tqdm

import math
from inspect import isfunction
from functools import partial

%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from einops import rearrange

import torch
from torch import nn, einsum
import torch.nn.functional as F

# **普及モデルとは？**
正規化フロー、$GAN$、$VAE$などの他の生成モデルと比較すると、（ノイズ除去）拡散モデルはそれほど複雑ではありません：

これらはすべて、ある単純な分布からノイズをデータサンプルに変換するものです。これはここでも同じで、 ニューラルネットワークが純粋なノイズから出発して徐々にデータをノイズ除去することを学習するのです。

イメージのためにもう少し詳しく説明すると、セットアップは2つのプロセスで構成されています。

- 固定された（あるいはあらかじめ定義された）任意の前方拡散プロセス $q$で，画像にガウスノイズを徐々に加え，最終的に純粋なノイズにする
- 学習型逆変換拡散処理 $p θ$ ここでニューラルネットワークは、純粋なノイズから始まり、実際の画像に行き着くまで、徐々に画像をノイズ除去するように学習されます。

<p align="center">
    <img src="https://drive.google.com/uc?id=1t5dUyJwgy2ZpDAqHXw7GhUAp2FE5BWHA" width="600" />
</p>

$t$でインデックスされる順方向および逆方向のプロセスは、ある有限時間ステップ数 $T$の間起こります（$DDPM$ の作者は $T= 1000$ を使用しています）。 $t= 0$でスタートし、データ分布から実画像 $x 0$ をサンプルし（$ImageNet$からの猫の画像としましょう）、前進過程は各タイムステップ $t$ でガウス分布からノイズをサンプルし、前のタイムステップの画像に追加しています。十分に大きな$T$と、各タイムステップでノイズを加えるための行儀の良いスケジュールがあれば、漸進的なプロセスによって、 $t= T$で [等方性ガウス分布](https://math.stackexchange.com/questions/1991961/gaussian-distribution-is-isotropic)と呼ばれるものが出来上がります。

より数学的な形式では
最終的には、ニューラルネットワークが最適化する必要のある、扱いやすい損失関数が必要なので、これをもっと正式に書きましょう。

させる  \\(q(\mathbf{x}_0)\\) は、実データの分布、例えば「実画像」の分布である。この分布からサンプリングして画像を得ることができる。  $x0∼q(x0)$ .ここで、前方拡散過程を定義する  $q(xt|xt-1)$ 各時間ステップでガウスノイズを加える  t 既知の分散スケジュールに従って  $0<β1<β2<$...$<βT<1$ かわりに
$q(xt|xt-1)=N(xt;1-βt-----√xt-1,βtI)$. 

正規分布（ガウス分布とも呼ばれる）は2つのパラメータで定義されることを思い出してください。  $μ$ と分散  $σ2≥0$ .基本的に、時間ステップにおけるそれぞれの新しい（少しノイズの多い）画像は  t が描画されます。 条件付ガウス分布をもって  $μt=1-βt-----√xt-1$ と  σ2t=βt をサンプリングすることによって行うことができます。  $ϵ ∼N(0,I)$ を設定し  $xt=1-βt-----√xt-1+βt--√ϵ$ .

$β  t$ は 各タイムステップで一定ではない ことに注意してくださいt (したがって添え字) --- 実際には、いわゆる 「分散スケジュール」を 定義 します。これは、これから見るように線形、二次、余弦などです (学習率スケジュールに少し似ています)。

からスタートするわけです。  x0 ということになり、結局は  $x1,...,xt,...,xT$で、ここで  $xT$ は、スケジュールを適切に設定すれば、純粋なガウスノイズとなる。

今、条件付き分布が分かっていれば  $p(xt-1|xt)$ というランダムなガウシアンノイズのサンプリングによって、このプロセスを逆に実行することができます。  $xT$ そして，それを徐々に「ノイズ除去」して，最終的に実際の分布からのサンプルを得ます。  $x0$.

しかし、私たちは知らない  $p(xt-1|xt)$ .この条件付き確率を計算するためには、すべての可能な画像の分布を知る必要があるため、難易度が高いのです。そこで、ニューラルネットワークを活用して じゅうきょひどうと呼ぶことにしましょう。  $pθ(xt-1|xt)$ である。  $θ$ はニューラルネットワークのパラメータであり、勾配降下法により更新される。

さて、それでは逆過程の（条件付き）確率分布を表現するためのニューラルネットワークが必要ですね。この逆過程もガウシアンだとすると、どんなガウシアン分布も2つのパラメータで定義されることを思い出してください。

によってパラメタライズされる平均値です。  $μθ$ ;
によってパラメタライズされる分散です。  $Σθ$ ;
ということで、そのプロセスをパラメトリックにすると
$$ p_\theta (\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \mu_\theta(\mathbf{x}_{t},t), \Sigma_\theta (\mathbf{x}_{t},t))$$
ここで、平均と分散もノイズレベルに条件付けされる  $t$ .

したがって、ニューラルネットワークは平均と分散を学習・表現する必要があります。しかし、DDPMの作者は分散を固定 し、 この条件付き確率分布の平均値のみを学習（表現） させることにした。論文より
> First, we set \\(\Sigma_\theta ( \mathbf{x}_t, t) = \sigma^2_t \mathbf{I}\\) to untrained time dependent constants. Experimentally, both \\(\sigma^2_t = \beta_t\\) and \\(\sigma^2_t  = \tilde{\beta}_t\\) (see paper) had similar results. 


まず 、 未調教の時間依存定数に 、$Sigma_theta$${x}_t$, $t$ = \sigma^2_t \mathbf{I}}) を設定します。 実験的には 、♪ \(sigma^2_t = \beta_t) と ♪ \(sigma^2_t = \tilde{beta}_t) (論文参照)の両方で同様の 結果が得られています。

これはその後、Improved diffusion modelsの論文で改良され、ニューラルネットワークが平均以外に、この逆行列の分散も学習するようになりました。

そこで、ニューラルネットワークはこの条件付き確率分布の平均値だけを学習・表現すればよいと仮定して、話を続ける。

目的関数の定義（平均値の再パラメタイズによる）
後方プロセスの平均を学習するための目的関数を導出するために、著者 らは\(q\) と \(p_theta\) の組み合わせが variational auto-encoder (VAE) として 見られることを観察 します(Kingma et al., 2013)。したがって、変分下界 （ELBOともいう）は、グランドトゥルースデータサンプル \(\mathbf{x}_0}) に関する負の対数尤度を最小化するために使用できる （ELBOに関する詳細は、VAE論文を参照すること） 。このプロセスの ELBOは、各タイムステップでの損失 和(L = L_0 + L_1 + ... + L_Thesis) であることが判明しました。Forward \(q) process and backward processの 構成から 、 損失の 各項(not for \(L_0 001))は実は 2つのガウス分布間のKL divergenceで、明示的に平均に関するL2損失として書くことができます!

Sohl-Dickstein et al.が示した ように、構築された前進過 程(forward process)の直接的な 結果として、( sums of Gaussians is also Gaussian) \(\mathbf{x}_0) を条件として任意のノイズレベルで サンプリング できるのです。これは とても便利な ことで、" Sample \(\mathbf{x}_t}" のために、 何度も "the\(q) " を 適用 する必要がありません。 q(\mathbf{x}_t | \mathbf{x}_0) = \cal{N}(\mathbf{x}_t; \sqrt{Thinbar{Thinalpha}_t}) であることが分かります。\mathbf{x}_0, (1- \bar{alpha}_t) \mathbf{I})$$.

with \(\alpha_t := 1 - \beta_t\) and \(\bar{alpha}t := \Pi_{s=1}^{t} \alpha_s :).この式を "nice property "と呼ぶことにしましょう。 これは、ガウスノイズをサンプリングして、それを適当にスケーリングして 、\(\mathbf{x}_0}) に加え れば、 そのまま \(\mathbf{x}_t})が得られることを意味します。 なお 、この\(Ⓐ) は既知の 分散スケジュール である \(Ⓐ) の 関数 なので、 これも既知で事前に計算 することが可能です。これにより、学習中に 損失関 数(loss function)のランダムな項を最適化 することができます ( 言い換えれば、 学習中に ランダムに (tenta)を サンプリングして、 (L_tenta)を 最適化 することができます)。

この特性のもう一つの利点は、Ho et al.は、(いくつかの数学の後、読者にこの素晴らしいブログ記事を参照してもらっていますが)代わりに平均を再パラメトリック化して、ニューラルネットワークに追加されたノイズを学習(予測)させることができます(via a network \(mathbf{epsilon}_theta(\mathbf{x}_t,を学習し、損失を構成するKL項に ノイズ レベルⒶ(t)Ⓑ が 加わることを予測します。つまり、このニューラルネットワークは（直接的な）平均予測器ではなく、ノイズ予測器となる。平均は次のように計算される。

Ъ⑅⑅⑅⑅⑅⑅⑅⑅⑅⑅⑅⑅⑅⑅ㄘ\left( \mathbf{x}_t - \frac{beta_t}{Thanksqrt{1- \bar{alpha}_t}} {Thanksqrt{1- | }} )\mathbf{mu}_theta(\mathbf{x}_t, t) \right)$$$\mathbf{mu}_theta(\mathbf{x}_t, t) = \frac{1}{sqrt{1}{alpha_t}} (\mathbf{x}_t, t)\left( \mathbf{x}_t - \frac{beta_t}{Thanksqrt{1- \bar{alpha}_t}} {Thanksqrt{1- | }} )\mathbf{epsilon}_theta(\mathbf{x}_t, t) \right)$$.

最終的な目的関数  Lt の場合、次のようになります（ランダムな時間ステップの場合）。  t 所定  ϵ∼N(0,I) ):

ここで 、${x}_0$ は初期（実画像、無汚染）画像で 、 固定前進プロセス で与えられた 直接ノイズレベル $t$である。${epsilon}$ は時間ステップでサンプリングされた純粋なノイズ であり、 [\(mathbf{epsilon}_theta (\mathbf{x}_t, t)\] は我々のニューラルネットである。 ニューラルネットワークは、ガウシアンノイズと予測値の平均二乗誤差(MSE)を用いて最適化されています。

これで学習アルゴリズムは次のようになる。

<p align="center">
    <img src="https://drive.google.com/uc?id=1LJsdkZ3i1J32lmi9ONMqKFg5LMtpSfT4" width="400" />
</p>

つまりは

われわれは無作為にサンプルを取る  x0 実際の未知で複雑な可能性のあるデータ分布から  q(x0) 
ノイズレベルをサンプリングする  t 満遍なく  1 と  T (を使用します（例：ランダムな時間ステップ）。
ガウス分布からノイズをサンプリングし，このノイズによって入力をレベル  t 上記で定義した nice プロパティを使って
このノイズを予測するようにニューラルネットワークを学習させる。  xt に適用されるノイズです。  $x0$ 予定に基づ$く  βt$ 
実際には、確率的勾配降下法を用いてニューラルネットワークを最適化するため、これらの作業はすべてデータのバッチで行われる。

ニューラルネット
ニューラルネットワークは特定の時間ステップでノイズの入った画像を取り込み、予測されたノイズを返す必要がある。予測されるノイズは入力画像と同じサイズ/解像度を持つテンソルであることに注意されたい。つまり、技術的には同じ形のテンソルを取り込み、出力するネットワークである。これにはどのようなニューラルネットワークを使えばよいのだろうか。

ここで典型的に使われているのは、典型的な「深層学習入門」のチュートリアルで覚えているであろう、オートエンコーダーと非常によく似たものです。オートエンコーダーは、エンコーダーとデコーダーの間に、いわゆる「ボトルネック」層があります。エンコーダーはまず画像を「ボトルネック」と呼ばれる小さな隠れ表現にエンコードし、次にデコーダーがその隠れ表現を実際の画像にデコードして戻す。これにより、ネットワークはボトルネック層に最も重要な情報のみを保持するようになる。

アーキテクチャの面では、DDPMの作者は、（Ronnebergerら、2015）（当時、医療画像のセグメンテーションのために最先端の結果を達成した）によって導入されたU-Netを採用した。このネットワークは、他のオートエンコーダーと同様に、ネットワークが最も重要な情報のみを学習するようにする中央のボトルネックで構成されています。重要なのは、エンコーダーとデコーダーの間に残留接続を導入し、勾配流を大幅に改善したことです（He et al., 2015のResNetに着想を得ています）。<p align="center">
    <img src="https://drive.google.com/uc?id=1_Hej_VTgdUWGsxxIuyZACCGjpbCGIUi6" width="400" />
</p>

このように、U-Netモデルは、まず入力をダウンサンプリング（空間分解能を小さくする）し、その後、アップサンプリングを行う。

以下、順を追って、このネットワークを実装していきます。

ネットワークヘルパー
まず、ニューラルネットワークを実装する際に使用されるいくつかのヘルパー関数とクラスを定義します。重要なのはResidualモジュールを定義することで、これは単に特定の関数の出力に入力を追加する（言い換えれば、特定の関数に残差接続を追加する）だけです。

また、アップサンプリングとダウンサンプリングの操作の別名を定義しています。

In [ ]:
def exists(x):
    return x is not None

def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        return self.fn(x, *args, **kwargs) + x

def Upsample(dim):
    return nn.ConvTranspose2d(dim, dim, 4, 2, 1)

def Downsample(dim):
    return nn.Conv2d(dim, dim, 4, 2, 1)

ポジションエンベッディング
ニューラルネットワークのパラメータは時間（ノイズレベル）にわたって共有されるので、著者らは正弦波位置エンコーディングを採用して、  t  、Transformer（Vaswani et al., 2017）に触発されてエンコード しています。これにより、ニューラルネットワークは、バッチ内のすべての画像について、どの特定の時間ステップ（ノイズレベル）で動作しているのかを「知る」ことができます。

SinusoidalPositionEmbeddingsモジュールは、形状（batch_size, 1）のテンソルを入力として受け取り（つまり、バッチ内のいくつかのノイズ画像のノイズレベル）、これを形状（batch_size, dim）のテンソルに変換します（dimは位置埋め込みの次元数です）。これは、後に見るように、各残差ブロックに追加される。

In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

ResNet/ConvNeXTブロック
次に、U-Netモデルのコアビルディングブロックを定義します。DDPMの著者らはWide ResNetブロック（Zagoruyko et al., 2016）を採用しましたが、Phil Wangは、後者が画像ドメインで大きな成功を収めたことから、ConvNeXTブロック（Liu et al., 2022）のサポートも追加することにしました。最終的なU-Netアーキテクチャにおいて、どちらかを選択することができます。

In [ ]:
class Block(nn.Module):
    def __init__(self, dim, dim_out, groups = 8):
        super().__init__()
        self.proj = nn.Conv2d(dim, dim_out, 3, padding = 1)
        self.norm = nn.GroupNorm(groups, dim_out)
        self.act = nn.SiLU()

    def forward(self, x, scale_shift = None):
        x = self.proj(x)
        x = self.norm(x)

        if exists(scale_shift):
            scale, shift = scale_shift
            x = x * (scale + 1) + shift

        x = self.act(x)
        return x

class ResnetBlock(nn.Module):
    """https://arxiv.org/abs/1512.03385"""
    
    def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=8):
        super().__init__()
        self.mlp = (
            nn.Sequential(nn.SiLU(), nn.Linear(time_emb_dim, dim_out))
            if exists(time_emb_dim)
            else None
        )

        self.block1 = Block(dim, dim_out, groups=groups)
        self.block2 = Block(dim_out, dim_out, groups=groups)
        self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb=None):
        h = self.block1(x)

        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb)
            h = rearrange(time_emb, "b c -> b c 1 1") + h

        h = self.block2(h)
        return h + self.res_conv(x)
    
class ConvNextBlock(nn.Module):
    """https://arxiv.org/abs/2201.03545"""

    def __init__(self, dim, dim_out, *, time_emb_dim=None, mult=2, norm=True):
        super().__init__()
        self.mlp = (
            nn.Sequential(nn.GELU(), nn.Linear(time_emb_dim, dim))
            if exists(time_emb_dim)
            else None
        )

        self.ds_conv = nn.Conv2d(dim, dim, 7, padding=3, groups=dim)

        self.net = nn.Sequential(
            nn.GroupNorm(1, dim) if norm else nn.Identity(),
            nn.Conv2d(dim, dim_out * mult, 3, padding=1),
            nn.GELU(),
            nn.GroupNorm(1, dim_out * mult),
            nn.Conv2d(dim_out * mult, dim_out, 3, padding=1),
        )

        self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb=None):
        h = self.ds_conv(x)

        if exists(self.mlp) and exists(time_emb):
            assert exists(time_emb), "time embedding must be passed in"
            condition = self.mlp(time_emb)
            h = h + rearrange(condition, "b c -> b c 1 1")

        h = self.net(h)
        return h + self.res_conv(x)

アテンションモジュール
次に、DDPMの作者が畳み込みブロックの間に追加したアテンションモジュールを定義します。アテンションは、NLPやビジョンからタンパク質折り畳みまで、AIの様々な領域で大きな成功を収めた有名なTransformerアーキテクチャ（Vaswani et al.、2017）のビルディングブロックです。Phil Wangは注意の2つの変種を採用しています：1つは通常のマルチヘッド自己注意（Transformerで使用）、もう1つは線形注意変種）、その時間およびメモリ要件は、通常の注意の2次関数に対して、シーケンス長に線形にスケールします。

アテンションメカニズムの詳細については、ジェイ・アラマー氏の素晴らしいブログ記事を参照してください。

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )
        q = q * self.scale

        sim = einsum("b h d i, b h d j -> b h i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("b h i j, b h d j -> b h i d", attn, v)
        out = rearrange(out, "b h (x y) d -> b (h d) x y", x=h, y=w)
        return self.to_out(out)

class LinearAttention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)

        self.to_out = nn.Sequential(nn.Conv2d(hidden_dim, dim, 1), 
                                    nn.GroupNorm(1, dim))

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )

        q = q.softmax(dim=-2)
        k = k.softmax(dim=-1)

        q = q * self.scale
        context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

        out = torch.einsum("b h d e, b h d n -> b h e n", context, q)
        out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)
        return self.to_out(out)

グループ正規化
DDPMの作者は、U-Netの畳み込み層／注意層をグループノーマライズでインターリーブしています（Wu et al.、2018）。以下では、PreNormクラスを定義し、このクラスは、さらに見るように、注意層の前にgroupnormを適用するために使用されます。なお、トランスフォーマーでは、注目の前と後のどちらに正規化を適用するかという議論があります。

In [ ]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.GroupNorm(1, dim)

    def forward(self, x):
        x = self.norm(x)
        return self.fn(x)

条件付きU-NET
さて、すべてのビルディングブロック（位置埋め込み、ResNet/ConvNeXTブロック、注意、グループ正規化）を定義したところで、ニューラルネットワーク全体を定義することになります。ネットワークの仕事は  ϵθ(xt,t) は、ノイズの多い画像＋ノイズレベルを一括して取り込み、入力に加えられたノイズを出力するものです。より正式には

ネットワークは、形状(batch_size, num_channels, height, width)と形状(batch_size, 1) のノイズレベルのバッチを入力として受け取り、形状(batch_size, num_channels, height, width) のテンソルを返します。
ネットワークは以下のように構築されています。

まず、ノイズの多い画像に対して畳み込み層を適用し、ノイズレベルに対する位置埋め込みを計算する
次に、一連のダウンサンプリングステージが適用される。各ダウンサンプリングステージは、2つのResNet/ConvNeXTブロック + groupnorm + attention + residual connection + ダウンサンプリングオペレーションから構成されます。
は、ResNet または ConvNeXT ブロックが適用され、注意とインターリーブされる。
次に、一連のアップサンプリングステージが適用される。各アップサンプリングステージは、2つのResNet/ConvNeXTブロック + groupnorm + attention + residual connection + upsample operationで構成されます。
最後に、ResNet/ConvNeXTブロックと畳み込み層が適用されます。
最終的にニューラルネットワークは、レゴブロックのように層を積み上げていきます（ただし、その仕組みを理解することが重要です）。

In [ ]:
class Unet(nn.Module):
    def __init__(
        self,
        dim,
        init_dim=None,
        out_dim=None,
        dim_mults=(1, 2, 4, 8),
        channels=3,
        with_time_emb=True,
        resnet_block_groups=8,
        use_convnext=True,
        convnext_mult=2,
    ):
        super().__init__()

        # 次元の決定
        self.channels = channels

        init_dim = default(init_dim, dim // 3 * 2)
        self.init_conv = nn.Conv2d(channels, init_dim, 7, padding=3)

        dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
        in_out = list(zip(dims[:-1], dims[1:]))
        
        if use_convnext:
            block_klass = partial(ConvNextBlock, mult=convnext_mult)
        else:
            block_klass = partial(ResnetBlock, groups=resnet_block_groups)

        # 埋め込み時間
        if with_time_emb:
            time_dim = dim * 4
            self.time_mlp = nn.Sequential(
                SinusoidalPositionEmbeddings(dim),
                nn.Linear(dim, time_dim),
                nn.GELU(),
                nn.Linear(time_dim, time_dim),
            )
        else:
            time_dim = None
            self.time_mlp = None

        # 層
        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])
        num_resolutions = len(in_out)

        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions - 1)

            self.downs.append(
                nn.ModuleList(
                    [
                        block_klass(dim_in, dim_out, time_emb_dim=time_dim),
                        block_klass(dim_out, dim_out, time_emb_dim=time_dim),
                        Residual(PreNorm(dim_out, LinearAttention(dim_out))),
                        Downsample(dim_out) if not is_last else nn.Identity(),
                    ]
                )
            )

        mid_dim = dims[-1]
        self.mid_block1 = block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)
        self.mid_attn = Residual(PreNorm(mid_dim, Attention(mid_dim)))
        self.mid_block2 = block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)

        for ind, (dim_in, dim_out) in enumerate(reversed(in_out[1:])):
            is_last = ind >= (num_resolutions - 1)

            self.ups.append(
                nn.ModuleList(
                    [
                        block_klass(dim_out * 2, dim_in, time_emb_dim=time_dim),
                        block_klass(dim_in, dim_in, time_emb_dim=time_dim),
                        Residual(PreNorm(dim_in, LinearAttention(dim_in))),
                        Upsample(dim_in) if not is_last else nn.Identity(),
                    ]
                )
            )

        out_dim = default(out_dim, channels)
        self.final_conv = nn.Sequential(
            block_klass(dim, dim), nn.Conv2d(dim, out_dim, 1)
        )

    def forward(self, x, time):
        x = self.init_conv(x)

        t = self.time_mlp(time) if exists(self.time_mlp) else None

        h = []

        # 低解像度処理
        for block1, block2, attn, downsample in self.downs:
            x = block1(x, t)
            x = block2(x, t)
            x = attn(x)
            h.append(x)
            x = downsample(x)

        # ボトルネック
        x = self.mid_block1(x, t)
        x = self.mid_attn(x)
        x = self.mid_block2(x, t)

        # 高解像度処理
        for block1, block2, attn, upsample in self.ups:
            x = torch.cat((x, h.pop()), dim=1)
            x = block1(x, t)
            x = block2(x, t)
            x = attn(x)
            x = upsample(x)

        return self.final_conv(x)

denoise_modelは上で定義したU-Netになります。我々は、真のノイズと予測されたノイズの間にHuberロスを採用することにします。

PyTorchデータセットの定義 + DataLoader
ここでは、通常のPyTorchデータセットを定義します 。このデータセットは、Fashion-MNIST, CIFAR-10, ImageNet などの実データセットからの画像を 、[-1,1] に直線的にスケーリングしたものから単純に構成されて います。

各画像は同じサイズにリサイズされます。面白いのは、画像もランダムに水平方向に反転されることです。論文より

CIFAR10では、学習時にランダムな水平フリップを使用しました。フリップを使用した場合と使用しない場合の両方を試した結果、フリップを使用することでサンプルの品質が若干向上することが分かりました。

ここでは、🤗Datasetsライブラリを用いて、Fashion MNISTデータセットをハブから簡単に読み込むことができます。このデータセットは、既に同じ解像度、すなわち28x28を持つ画像から構成されています。

In [ ]:
def cosine_beta_schedule(timesteps, s=0.008):
    """
    cosine schedule as proposed in https://arxiv.org/abs/2102.09672
    """
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.9999)

def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

def quadratic_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start**0.5, beta_end**0.5, timesteps) ** 2

def sigmoid_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    betas = torch.linspace(-6, 6, timesteps)
    return torch.sigmoid(betas) * (beta_end - beta_start) + beta_start

まず始めに、リニアスケジュールで  $T=200$ の時間ステップと様々な変数を定義します。  $βt$ というように、分散の累積積が必要です。  $α¯t$ .以下の各変数は単なる1次元のテンソルであり、以下の値を格納する。  $t$ まで  $T$ .また、重要なことに、我々は 抜粋関数を使用することで、適切な  $t$ のインデックスを一括で取得することができます。

In [ ]:
timesteps = 200

# β線スケジュールを定義する
betas = linear_beta_schedule(timesteps=timesteps)

#　アルファを定義する
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)

# 拡散 q(x_t | x_{t-1}) などの計算を行う
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# 事後q(x_{t-1} | x_t, x_0)の計算結果
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

def extract(a, t, x_shape):
    batch_size = t.shape[0]
    out = a.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

We'll illustrate with a cats image how noise is added at each time step of the diffusion process.

In [ ]:
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
image

<img src="https://drive.google.com/uc?id=17FXnvCTl96lDhqZ_io54guXO8hM-rsQ2" width="400" />

ノイズはPillow Imageではなく、PyTorchのテンソルに加えられる。まず、PIL画像からPyTorchテンソル（その上にノイズを加えることができる）へ、またはその逆を可能にする画像変換を定義します。

これらの変換は非常に簡単で、まず画像を正規化するために  255 (にあるような）。  $[0,1]$の範囲）であることを確認し、それらが  $[-1,1]$ の範囲です。DPPMの論文より。

の整数からなる画像データを想定しています。  ${0,1,...,255}$ に線形に拡大される。  $[-1,1]$ .これにより、ニューラルネットワークの反転処理は、標準的な正規の事前分布から始まる一貫したスケールの入力で動作することが保証されます。  $p(xT)$ . 


In [ ]:
from torchvision.transforms import Compose, ToTensor, Lambda, ToPILImage, CenterCrop, Resize

image_size = 128
transform = Compose([
    Resize(image_size),
    CenterCrop(image_size),
    ToTensor(), # HWC 形式の Numpy 配列に変換し、255 で割る。
    Lambda(lambda t: (t * 2) - 1),
    
])

x_start = transform(image).unsqueeze(0)
x_start.shape

<div class="output stream stdout">

    Output:
    ----------------------------------------------------------------------------------------------------
    torch.Size([1, 3, 128, 128])

</div>

We also define the reverse transform, which takes in a PyTorch tensor containing values in $[-1, 1]$ and turn them back into a PIL image:

In [ ]:
import numpy as np

reverse_transform = Compose([
     Lambda(lambda t: (t + 1) / 2),
     Lambda(lambda t: t.permute(1, 2, 0)), # CHW to HWC
     Lambda(lambda t: t * 255.),
     Lambda(lambda t: t.numpy().astype(np.uint8)),
     ToPILImage(),
])

Let's verify this:

In [ ]:
reverse_transform(x_start.squeeze())

<img src="https://drive.google.com/uc?id=1WT22KYvqJbHFdYYfkV7ohKNO4alnvesB" width="100" />

ここで、論文と同様に前方拡散過程を定義することができる。


In [ ]:
# forward diffusion
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x_start.shape
    )

    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

Let's test it on a particular time step:

In [ ]:
def get_noisy_image(x_start, t):
  # ノイズを加える
  x_noisy = q_sample(x_start, t=t)

  # PILの画像に戻す
  noisy_image = reverse_transform(x_noisy.squeeze())

  return noisy_image

In [ ]:
# 時間がかかる
t = torch.tensor([40])

get_noisy_image(x_start, t)

<img src="https://drive.google.com/uc?id=1Ra33wxuw3QxPlUG0iqZGtxgKBNdjNsqz" width="100" />

Let's visualize this for various time steps:

In [ ]:
import matplotlib.pyplot as plt

# 再現性のためシードを固定する
torch.manual_seed(0)

# source: https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
def plot(imgs, with_orig=False, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # 1列でも2次元のグリッドを作成する
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(figsize=(200,200), nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [image] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

In [ ]:
plot([get_noisy_image(x_start, torch.tensor([t])) for t in [0, 50, 100, 150, 199]])

<img src="https://drive.google.com/uc?id=1QifsBnYiijwTqru6gur9C0qKkFYrm-lN" width="800" />
    
つまり、モデルが与えられた損失関数を次のように定義することができるようになったのである。

In [ ]:
def p_losses(denoise_model, x_start, t, noise=None, loss_type="l1"):
    if noise is None:
        noise = torch.randn_like(x_start)

    x_noisy = q_sample(x_start=x_start, t=t, noise=noise)
    predicted_noise = denoise_model(x_noisy, t)

    if loss_type == 'l1':
        loss = F.l1_loss(noise, predicted_noise)
    elif loss_type == 'l2':
        loss = F.mse_loss(noise, predicted_noise)
    elif loss_type == "huber":
        loss = F.smooth_l1_loss(noise, predicted_noise)
    else:
        raise NotImplementedError()

    return loss

前方拡散プロセスの定義
前方拡散処理は、実分布から時間ステップ数で徐々に画像にノイズを加える  T  . これは分散スケジュールに従って行わ れる。オリジナルのDDPMの作者は、線形スケジュールを採用していました。

から線形に増加する定数に設定した。  β1=10-4  まで  βT=0.02 .

しかし、(Nichol et al., 2021)では、コサインスケジュールを採用するとより良い結果が得られることが示された。

以下、様々なスケジュールを定義していきます。  T のタイムステップと、それに対応する累積分散のような変数が必要です。

In [ ]:
from datasets import load_dataset

# ハブからデータセットを読み込む
dataset = load_dataset("fashion_mnist")
image_size = 28
channels = 1
batch_size = 128

次に、データセット全体にオンザフライで適用する関数を定義します。これには with_transform 関数を使います。この関数は基本的な画像の前処理を行うだけです。ランダムに水平反転させ、スケールを変え、最後に$[-1,1]$の範囲に値を持つようにします。

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

# 画像変換を定義する (例: torchvision を使用)
transform = Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Lambda(lambda t: (t * 2) - 1)
])

# 関数の定義
def transforms(examples):
   examples["pixel_values"] = [transform(image.convert("L")) for image in examples["image"]]
   del examples["image"]

   return examples

transformed_dataset = dataset.with_transform(transforms).remove_columns("label")

# データ読み込み機の作成
dataloader = DataLoader(transformed_dataset["train"], batch_size=batch_size, shuffle=True)

In [ ]:
batch = next(iter(dataloader))
print(batch.keys())

<div class="output stream stdout">

    Output:
    ----------------------------------------------------------------------------------------------------
    dict_keys(['pixel_values'])

</div>

# **サンプリング**
学習中にモデルからサンプリングする（進捗を把握するため）ので、そのためのコードを以下に定義します。サンプリングは論文中ではアルゴリズム2としてまとめられている。

<img src="https://drive.google.com/uc?id=1ij80f8TNBDzpKtqHjk_sh8o5aby3lmD7" width="500" />

拡散モデルから新しい画像を生成するには、拡散プロセスを反転させる必要があります。  T ここで、ガウス分布から純粋なノイズをサンプリングし、ニューラルネットワークを使って（学習した条件付き確率を使って）徐々にノイズを除去し、最終的に時間ステップ  t=0 .このように、より少ないノイズ除去率で画像を得ることができます。  xt-1 ノイズ予測器を用いて、平均の再パラメータ化を行うことで、このようなことが可能になります。分散は前もってわかっていることを忘れないでください。

理想は、実際のデータ分布から得られたような画像に仕上げることです。

これを実装したのが以下のコードです。

In [ ]:
@torch.no_grad()
def p_sample(model, x, t, t_index):
    betas_t = extract(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, x.shape)
    
    # 論文中の式11
    # 我々のモデル（ノイズ予測器）を使って平均を予測する
    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model(x, t) / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        return model_mean
    else:
        posterior_variance_t = extract(posterior_variance, t, x.shape)
        noise = torch.randn_like(x)
        # アルゴリズム2行目 4．
        return model_mean + torch.sqrt(posterior_variance_t) * noise 

# アルゴリズム2ですが、全画像を保存します。
@torch.no_grad()
def p_sample_loop(model, shape):
    device = next(model.parameters()).device

    b = shape[0]
    # 純粋なノイズからスタート（バッチ内の各例で）
    img = torch.randn(shape, device=device)
    imgs = []
    
    for i in tqdm(reversed(range(0, timesteps)), desc='sampling loop time step', total=timesteps):
        img = p_sample(model, img, torch.full((b,), i, device=device, dtype=torch.long), i)
        imgs.append(img.cpu().numpy())
    return imgs

@torch.no_grad()
def sample(model, image_size, batch_size=16, channels=3):
    return p_sample_loop(model, shape=(batch_size, channels, image_size, image_size))

上記のコードはオリジナルの実装を単純化したものであることに注意してください。私たちは、私たちの簡略化したコード（論文のアルゴリズム2に沿ったもの）が、オリジナルの複雑な実装と同じように動作することを発見しました。

モデルの学習
次に、通常のPyTorchの方法でモデルを学習させます。また、上記で定義したサンプルメソッドを用いて、生成された画像をペリディオカルに保存するためのロジックをいくつか定義します。

In [ ]:
from pathlib import Path

def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

results_folder = Path("./results")
results_folder.mkdir(exist_ok = True)
save_and_sample_every = 1000

Below, we define the model, and move it to the GPU. We also define a standard optimizer (Adam).

In [ ]:
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Unet(
    dim=image_size,
    channels=channels,
    dim_mults=(1, 2, 4,)
)
model.to(device)

optimizer = Adam(model.parameters(), lr=1e-3)

Let's start training!

In [ ]:
from torchvision.utils import save_image

epochs = 5

for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
      optimizer.zero_grad()

      batch_size = batch["pixel_values"].shape[0]
      batch = batch["pixel_values"].to(device)

      # アルゴリズム1 行目：バッチ内のすべての例に対して一様にtをサンプリングする
      t = torch.randint(0, timesteps, (batch_size,), device=device).long()

      loss = p_losses(model, batch, t, loss_type="huber")

      if step % 100 == 0:
        print("Loss:", loss.item())

      loss.backward()
      optimizer.step()

      # 生成された画像の保存
      if step != 0 and step % save_and_sample_every == 0:
        milestone = step // save_and_sample_every
        batches = num_to_groups(4, batch_size)
        all_images_list = list(map(lambda n: sample(model, batch_size=n, channels=channels), batches))
        all_images = torch.cat(all_images_list, dim=0)
        all_images = (all_images + 1) * 0.5
        save_image(all_images, str(results_folder / f'sample-{milestone}.png'), nrow = 6)

<div class="output stream stdout">

    Output:
    ----------------------------------------------------------------------------------------------------
    Loss: 0.46477368474006653
    Loss: 0.12143351882696152
    Loss: 0.08106148988008499
    Loss: 0.0801810547709465
    Loss: 0.06122320517897606
    Loss: 0.06310459971427917
    Loss: 0.05681884288787842
    Loss: 0.05729678273200989
    Loss: 0.05497899278998375
    Loss: 0.04439849033951759
    Loss: 0.05415581166744232
    Loss: 0.06020551547408104
    Loss: 0.046830907464027405
    Loss: 0.051029372960329056
    Loss: 0.0478244312107563
    Loss: 0.046767622232437134
    Loss: 0.04305662214756012
    Loss: 0.05216279625892639
    Loss: 0.04748568311333656
    Loss: 0.05107741802930832
    Loss: 0.04588869959115982
    Loss: 0.043014321476221085
    Loss: 0.046371955424547195
    Loss: 0.04952816292643547
    Loss: 0.04472338408231735

</div>

サンプリング（推論）
モデルからサンプリングするには、上で定義したsample関数を使えばいいのです。

In [ ]:
# サンプル画像64枚
samples = sample(model, image_size=image_size, batch_size=64, channels=channels)

In [ ]:
# その中から1枚表示する
random_index = 5
plt.imshow(samples[-1][random_index].reshape(image_size, image_size, channels), cmap="gray")

<img src="https://drive.google.com/uc?id=1ytnzS7IW7ortC6ub85q7nud1IvXe2QTE" width="300" />

どうやら、いい感じのTシャツを生成できるようです！なお、今回学習させたデータセットは、かなり低解像度（28x28）であることに留意してください。

また、ノイズ除去の過程をGIFで作成することもできます。

In [ ]:
import matplotlib.animation as animation

random_index = 53

fig = plt.figure()
ims = []
for i in range(timesteps):
    im = plt.imshow(samples[i][random_index].reshape(image_size, image_size, channels), cmap="gray", animated=True)
    ims.append([im])

animate = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
animate.save('diffusion.gif')
plt.show()

フォローアップの読み方
なお、DDPMの論文では、拡散モデルが（無）条件付き画像生成の有望な方向性であることが示されました。これはその後、（非常に）改善され、特にテキスト条件付き画像生成のために改善されました。以下に、いくつかの重要な（しかし網羅的ではない）後続の研究を列挙する。

Improved Denoising Diffusion Probabilistic Models(Nichol et al., 2021): 条件付き分布の分散を（平均以外に）学習することが性能向上に役立つことを発見した。
高忠実度画像生成のためのカスケード拡散モデル(Ho et al., 2021): 高忠実度画像合成のために解像度を上げる画像を生成する複数の拡散モデルのパイプラインからなるカスケード拡散を導入する
画像合成において拡散モデルがGANに勝る(Dhariwal et al., 2021): U-Netアーキテクチャを改善し、分類器ガイダンスを導入することで、拡散モデルが現在の最新鋭の生成モデルより優れた画像サンプル品質を達成できることを示す。
Classifier-Free Diffusion Guidance(Ho et al., 2021): 条件付き拡散モデルと無条件拡散モデルを単一のニューラルネットワークで共同学習することにより、拡散モデルのガイドに分類器が不要であることを示す。
CLIP Latentsによる階層的テキスト条件付き画像生成(DALL-E 2)(Ramesh et al., 2022): 事前にテキストキャプションをCLIP画像埋め込みに変換し、その後拡散モデルにより画像にデコードする
フォトリアリスティックなテキストから画像への拡散モデルと深い言語理解（ImageGen）（Sahariaら、2022）：事前に学習した大規模な言語モデル（T5など）とカスケード拡散を組み合わせることで、テキストから画像の合成にうまく機能することを示しています。
なお、このリストには、執筆時点（2022年6月7日）までの重要な作品しか含まれていません。

今のところ、拡散モデルの主な（おそらく唯一の）欠点は、画像を生成するために複数のフォワードパスが必要なことだと思われます（GANのような生成モデルではそうではありません）。しかし、わずか10ステップのノイズ除去で忠実度の高い画像を生成する研究が進められています。